In [3]:
import torch

# Import Data

In [4]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [5]:
print("Number of character in the dataset: ", len(text))

Number of character in the dataset:  1115394


In [6]:
#sample of dataset
print(text[:500])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


# Create Maps

In [7]:
#get all character in our dataset
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print("Number of unique chars: ", vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Number of unique chars:  65


In [8]:
# maps a character to index (integer)
stoi = {ch:i for i, ch in enumerate(chars)}

# maps a index to a character
itos = {i:ch for i, ch in enumerate(chars)}

#takes a string and outputs a array of index 
encode = lambda s : [stoi[ch] for ch in s]

#takes a array of index and outputs a string
decode = lambda l : ''.join([itos[i] for i in l])

#eg
print(encode("hello there"))
print(decode(encode("hello there")))

[46, 43, 50, 50, 53, 1, 58, 46, 43, 56, 43]
hello there


# Enode entire text into a tensor

In [9]:
data = torch.tensor(encode(text), dtype=torch.long)

print(data.shape, data.dtype)

#this is what the text looks like after encoding 
print(data[:500])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

# Train and validation split

In [10]:
#we will use 90% data for train and rest of 10% for test
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

# Creating Blocks

In [11]:
# we will be using blocks size of 8 to train. 
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [12]:
#what does block traing means:
#block size is the maximum context length

x = train_data[:block_size]
print("x: ", x)
y = train_data[1:block_size + 1]
print("y: ", y)
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target is: {target}")

x:  tensor([18, 47, 56, 57, 58,  1, 15, 47])
y:  tensor([47, 56, 57, 58,  1, 15, 47, 58])
when input is tensor([18]) the target is: 47
when input is tensor([18, 47]) the target is: 56
when input is tensor([18, 47, 56]) the target is: 57
when input is tensor([18, 47, 56, 57]) the target is: 58
when input is tensor([18, 47, 56, 57, 58]) the target is: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is: 58


# Creating Batches

In [13]:
torch.manual_seed(1337)
batch_size = 4 # how many training examples we will work with
block_size = 8 # Maximum context length

def get_batch(split):
    """
    input: split : train or test
    output: for each block give out input and target based on batch
    """

    data = train_data if split == "train" else val_data

    #pick random starting point and 
    #len(data) - block_size is the maximum because from that block we would like 8 len sequence
    ix = torch.randint(len(data)-block_size, (batch_size, )) # vector of len batch size
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size + 1] for i in ix])

    return x, y

In [14]:
# this xb and yb will be used though out the training for sanity check

xb, yb = get_batch('train')
print("inputs size: ", xb.shape)
print(xb)

print("Target size: ", yb.shape)
print(yb)

inputs size:  torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
Target size:  torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [15]:
#demostration how it trains

for b in range(batch_size): #batch dim
    print(f"block number {b}")
    for t in range(block_size): #context or block or time dim
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target is: {target}")
    print("")

block number 0
when input is [24] the target is: 43
when input is [24, 43] the target is: 58
when input is [24, 43, 58] the target is: 5
when input is [24, 43, 58, 5] the target is: 57
when input is [24, 43, 58, 5, 57] the target is: 1
when input is [24, 43, 58, 5, 57, 1] the target is: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target is: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target is: 39

block number 1
when input is [44] the target is: 53
when input is [44, 53] the target is: 56
when input is [44, 53, 56] the target is: 1
when input is [44, 53, 56, 1] the target is: 58
when input is [44, 53, 56, 1, 58] the target is: 46
when input is [44, 53, 56, 1, 58, 46] the target is: 39
when input is [44, 53, 56, 1, 58, 46, 39] the target is: 58
when input is [44, 53, 56, 1, 58, 46, 39, 58] the target is: 1

block number 2
when input is [52] the target is: 58
when input is [52, 58] the target is: 1
when input is [52, 58, 1] the target is: 58
when input is [52, 58, 1, 58] the

# Implement Bigram model

In [16]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

In [17]:
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__() # inherit all the functions from nn.Module 
        # embedding map vocab size x embedding dim
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets):
        # idx is going to be a certain number of batches (B)
        # for each batch it will contain certain length of context (T)
        # idx = (B, T) and it is going to be embedding to C dim
        logits = self.token_embedding_table(idx) # (B, T, C) 

        return logits

m = BigramLanguageModel(vocab_size)
out = m(xb, yb)
print("vocab_size: ", vocab_size)
print("xb shape: ", xb.shape)

#4 batches, 8 block or maximum context length, 65 embedding dim
print(out.shape)

vocab_size:  65
xb shape:  torch.Size([4, 8])
torch.Size([4, 8, 65])


In [18]:
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            ## this helps to feed into cross entropy 
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context 
        # max_new_token is how many prediction we make. We make prediction and 
        # concat it and use it to predict next 
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx) # feedforward 
            # focus only on the last time step 
            logits = logits[:, -1, :] # becomes (B, C) 
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            # use the probs to sample 
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.6630, grad_fn=<NllLossBackward0>)

v?vqKGh
txYXcDRSW&jxn;P Kuvbv
Bl:KuMWi;JYlFahlK3u.M,QV;oky3wcxoYPQpbpb
r3Nc 'MQHQ uOfxoxwXIvSE-$&a$K


In [19]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [20]:
batch_size = 32
for steps in range(100):
    # sample each time
    xb, yb = get_batch('train')

    # feedforward
    logits, loss = m(xb, yb)

    # backpropogation 
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

4.4756927490234375


In [21]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


h&uf tFRbdrNx:c
tMPvc r qDqzXw!JTfuFBKOqXFnHHvnG?TMpvws LbuBLbIwXeY -VaiaHoIOdRoeCgpNN.EG;Jut-VRe:dKGhxLIiaE!SvFh&TEDhZxasjC?sLHj?WQxqDzhbCOkc?ck'UJNJ;DbssGHqJgWADQ Uw KKoEJAtQidggCyyYfguPVkvpt!ZEEQVqDuINNVSEtJBYQxGNafLHoGS:hCgZGws''Jlajnzv3jflXNxEDlnHKoykS;Dh
QReCstiC-GYL!?mnzXqcckYz?XU3.u3uI'WBLtFlIW$,VQgwbxwRH
h:l-3cq-V3!H$jM:G EDH
W.aWfqjkacXs.X;J$hDtkx&vVsLIipvaJ,odyah:BrGd?XB!HnsnpcsLfLSlXcmHpbFnI$kKYqyX!xk:LIVxxeuWCk
d?sFHhSE$odgr'''MMUhmvVRHCo:L&TkKzmTnG onpc&esBbKyZfwISUhGY.,Fp&tqJnwjhD


# Trick of trianglar matrix 

In [22]:
# we have 8 tokens in the dataframe for each batch. 
# we want info flow from 1, 2 -> 3 but we want to info flow form 4 -> 1 or 2 or 3
torch.manual_seed(1337) 

B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [23]:
# example of info flowing to let say 3 is by average from 1 to 3. 
# we want x[b, t] = mean x[b, i] where i <= t
# naive method
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b, t] = torch.mean(xprev, 0)

In [24]:
# using matrix multiplication 
torch.manual_seed(42)
# create lower triangular matrix 
a = torch.tril(torch.ones(3, 3))
# create probabilities by summing across row
a = a / torch.sum(a, 1, keepdim=True)
# random int 
b = torch.randint(0, 10, (3, 2)).float()
# use upper triangular matrix a with b to get average
c = a @ b
# first 1 has only the average of 2 
# second is 2 + 6/2 = 4
# third is 2 + 6 + 6 / 3
a, b, c

(tensor([[1.0000, 0.0000, 0.0000],
         [0.5000, 0.5000, 0.0000],
         [0.3333, 0.3333, 0.3333]]),
 tensor([[2., 7.],
         [6., 4.],
         [6., 5.]]),
 tensor([[2.0000, 7.0000],
         [4.0000, 5.5000],
         [4.6667, 5.3333]]))

In [27]:
# using softmax 

# upper triangular matrix
tril = torch.tril(torch.ones(T, T))
# weights 
wei = torch.zeros((T,T))

# fill all the upper triangular part i.e. zeros to -inf 
# we are preparing it to exp
wei = wei.masked_fill(tril == 0, float('-inf'))
print(wei)
# apply soft max 
wei = F.softmax(wei, dim=-1)
print(wei)

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


In [28]:
# (T, T) @ (B, T, C) pytorch will convert it to 
# (B, T, T) @ (B, T, C) -> (B, T, C)
xbow2 = wei @ x 
xbow2.shape

torch.Size([4, 8, 2])

# Self Attention (Single Head)

In [31]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 32])

In [40]:
# Dimension for Query and Key
head_size = 16

# what the current token is looking for 
# treat it like a project matrix (C is the embedding of a token into dim C),
# then we use this matrix to find what the token is looking for by projecting into (dim head_size)
query = nn.Linear(C, head_size, bias=False)

# represents the information that each token offers.
key = nn.Linear(C, head_size, bias=False)

# the actual content or information a token provides
value = nn.Linear(C, head_size, bias=False)

print("query: ", query)
print("key: ", key)
print("value: ", value)

query:  Linear(in_features=32, out_features=16, bias=False)
key:  Linear(in_features=32, out_features=16, bias=False)
value:  Linear(in_features=32, out_features=16, bias=False)


In [43]:
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
# again you see it as embedding going from 32 dim to 16 dim 
k.shape, q.shape

(torch.Size([4, 8, 16]), torch.Size([4, 8, 16]))

In [44]:
# now we look for weight by checking each query against the key
# higher the query matches with key higher value
# we just do dot product for this
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

In [51]:
# want to make sure the only past infromation is used not the future 
tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

wei[0], wei.shape
# firstly we have 4 batches with 8 tokens 
# we want see the for the first token only thing that matters is th token itself 
# but for second nearly 40% of info from the first token matches with second token. 

(tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3955, 0.6045, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3099, 0.3160, 0.3741, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2359, 0.2359, 0.2672, 0.2610, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2199, 0.1939, 0.1961, 0.1950, 0.1950, 0.0000, 0.0000, 0.0000],
         [0.1710, 0.1709, 0.1635, 0.1632, 0.1641, 0.1672, 0.0000, 0.0000],
         [0.1399, 0.1400, 0.1403, 0.1449, 0.1405, 0.1525, 0.1420, 0.0000],
         [0.1230, 0.1228, 0.1229, 0.1245, 0.1235, 0.1235, 0.1238, 0.1359]],
        grad_fn=<SelectBackward0>),
 torch.Size([4, 8, 8]))

In [53]:
# we get the value of the token
v = value(x)

# Now we can use the weight to create a new token
# this much for than embedding of the token 
# it has the context of the other tokens that came before it 
# this much more enriched token 
out = wei @ v

out.shape

torch.Size([4, 8, 16])